<a href="https://colab.research.google.com/github/lelongc/ro/blob/main/tts_quenrVNpynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🚀 QWEN-TTS FINAL FIX (Đã thêm thư viện SOX)
import os
import sys
import subprocess
import time

print("🔄 Đang cài đặt lại từ đầu... (Mất khoảng 3 phút)")

# 1. Cài đặt công cụ hệ thống (OS Level)
subprocess.run("apt-get update -q", shell=True)
subprocess.run("apt-get install -y sox libsox-dev libsox-fmt-all ffmpeg", shell=True)

# 2. Cài đặt thư viện Python (Bổ sung 'sox' mà code trước bị thiếu)
# Đã thêm: sox, numpy<2 (để tránh xung đột)
subprocess.run("pip install -q sox torch torchaudio --index-url https://download.pytorch.org/whl/cu121", shell=True)
subprocess.run("pip install -q gradio soundfile librosa onnxruntime-gpu accelerate", shell=True)
subprocess.run("pip install -q git+https://github.com/huggingface/transformers.git", shell=True)

# 3. Tải mã nguồn Qwen-TTS
if not os.path.exists('/content/ComfyUI-Qwen-TTS'):
    print("📥 Đang tải mã nguồn Qwen-TTS...")
    subprocess.run("git clone https://github.com/flybirdxx/ComfyUI-Qwen-TTS.git /content/ComfyUI-Qwen-TTS", shell=True)

# 4. Cấu hình đường dẫn
repo_path = '/content/ComfyUI-Qwen-TTS'
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

# 5. Xóa cache import cũ
import importlib
importlib.invalidate_caches()

print("✅ Cài đặt thư viện xong! Đang khởi động AI...")

# --- PHẦN CHẠY ỨNG DỤNG ---
import torch
import soundfile as sf
import tempfile
import gradio as gr
import gc

# Import Module (Có xử lý lỗi)
try:
    from qwen_tts.inference.qwen3_tts_model import Qwen3TTSModel
except ImportError as e:
    # Nếu vẫn lỗi, thử cài lại sox trực tiếp và reload
    print(f"⚠️ Phát hiện lỗi thiếu thư viện: {e}. Đang tự động sửa...")
    subprocess.run("pip install sox", shell=True)
    import site
    site.main()
    from qwen_tts.inference.qwen3_tts_model import Qwen3TTSModel

# Cấu hình Model
current_model = None

def load_model():
    global current_model
    if current_model is not None:
        return current_model

    print("📥 Đang tải Model Qwen (1.7GB)... Lần đầu mất 2-3 phút!")
    # Tải model
    try:
        model = Qwen3TTSModel.from_pretrained(
            "Qwen/Qwen3-TTS-12Hz-1.7B-Base",
            torch_dtype=torch.float16,
            device_map="cuda:0",
            attn_implementation="sdpa"
        )
        current_model = model
        print("✅ Model đã sẵn sàng!")
        return model
    except Exception as e:
        print(f"❌ Lỗi tải model: {e}")
        return None

def voice_clone(text, audio, transcript):
    if not text or not audio: return None

    try:
        model = load_model()
        if not model: return "Lỗi: Không tải được model."

        prompt = model.create_voice_clone_prompt(
            ref_audio=audio,
            ref_text=transcript if transcript else None,
            x_vector_only_mode=True # Fast mode cho đỡ lỗi
        )

        with torch.inference_mode():
            audio_out, sr = model.generate_voice_clone(text=text, voice_clone_prompt=prompt)

        output_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        sf.write(output_file.name, audio_out[0], sr)
        return output_file.name
    except Exception as e:
        return f"Lỗi sinh giọng: {str(e)}"

# Giao diện
css = "body {background: #f0f2f5;} .container {max-width: 800px; margin: auto;}"
with gr.Blocks(title="Qwen Studio Fixed v2", css=css) as demo:
    gr.Markdown("# 🎙️ Qwen-TTS Voice Cloning (Final Fix)")

    with gr.Row():
        with gr.Column():
            inp_text = gr.Textbox(label="Văn bản (Text)", lines=5, placeholder="Enter text here...")
            inp_audio = gr.Audio(label="Giọng mẫu (Reference Audio - 5-10s)", type="filepath")
            inp_trans = gr.Textbox(label="Transcript giọng mẫu (Optional)")
            btn = gr.Button("🚀 Tạo Giọng", variant="primary")
        with gr.Column():
            out_audio = gr.Audio(label="Kết quả")

    btn.click(voice_clone, inputs=[inp_text, inp_audio, inp_trans], outputs=out_audio)

print("🔥 Hoàn tất! Bấm vào link Public URL bên dưới 👇")
demo.launch(share=True, debug=True)

🔄 Đang cài đặt lại từ đầu... (Mất khoảng 3 phút)
